In [1]:
import sys
import os
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt
from IPython.display import clear_output
from imageio import imread
from imageio import imwrite
%matplotlib inline

In [2]:
path_to_video = './SAT.mp4'

In [3]:
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
        
frame_save_path = './frames_A1/'
cap = cv2.VideoCapture(path_to_video)
create_dir_if_not_exists(frame_save_path)
create_dir_if_not_exists('./Motion/')
if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_counter = 0                                             # FRAME_COUNTER
while(1):
    return_flag, frame = cap.read()  
    if not return_flag:
        print('Video Reach End')
        break
    # Main Content - Start
    cv2.imshow('VideoWindowTitle-Ping Pang', frame)
    cv2.imwrite(frame_save_path + 'frame%d.tif' % frame_counter, frame)
    frame_counter += 1
    # Main Content - End    
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


Video Reach End


In [4]:
import math
import cv2
def arrowdraw(img, x1, y1, x2, y2, ssd):
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10
    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2
    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2
    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    
    r = 255
    g = 255
    b = 255
    width = 0
    if (ssd <= 75000):
        width = 1
    elif (ssd <= 10000):
        width = 2
    else:
        width = 3
        
    img = cv2.line(img, (x1, y1), (x2, y2), (r, g, b), width)
    img = cv2.line(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)),     (r, g, b), width)
    img = cv2.line(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)),     (r, g, b), width)
    
    return img

In [5]:
def split_into_block_coords(frame, block_width):
    blocks = []
    counter1 =0
    counter3 = 0
    counter2 = block_width
    counter4 = block_width
    #current_block = 0
    while counter4 <= frame1.shape[0]:
        while counter2 <=frame1.shape[1]:
            block = [[counter3, counter4],[counter1, counter2]]
            blocks.append(block)
            #if (current_block = 27):
            #    print(counter3, counter4, counter1, counter2)
            counter2 += block_width
            counter1 += block_width
            #current_block += 1
        counter1 = 0
        counter2 = block_width
        counter3 += block_width
        counter4 += block_width
    return blocks

In [6]:
def get_neighbour_coords(coords, block_width, levels, max_y, max_x):
    if (levels == 0):
        return coords
    
    neighbours = []
    x = [coords[0][0] - block_width*levels, coords[0][1] - block_width*levels]
    y = [coords[1][0] - block_width*levels, coords[1][1] - block_width*levels]
    
    #print(x, y)
    counter = 0
    # Get all adjacent coords
    for i in range(levels*2 +1):
        for j in range(levels*2 +1):
            #neighbours.append( [[y[0]+(counter*block_width), y[1]+(counter*block_width)], x])
            newcoord = [[y[0] + counter*block_width, y[1] + counter*block_width], [x[0], x[1]]]
            counter += 1
            neighbours.append(newcoord)
        x[0] += block_width
        x[1] += block_width
        counter = 0
    return_list = []
    valid = 1
    #Remove out of bounds coordinates
    for i in range(len(neighbours)):
        #print(neighbours[i], max_x, max_y)
        if (neighbours[i][0][0] < 0 or neighbours[i][1][0] < 0) :
            valid = 0
        if (neighbours[i][0][1] > max_y or neighbours[i][1][1] > max_x):
            valid = 0
        if (valid == 1):
            return_list.append(neighbours[i])
        valid = 1
    return return_list

In [7]:
import math
def computing_min_ssd(neighbours, coord, frame1, frame2, width, threshold):
    lowest_ssd = math.inf
    match = None
    A = frame1[coord[0][0]:coord[0][1], coord[1][0]:coord[1][1]]
    for neighbour in neighbours:
        B = frame2[neighbour[0][0]:neighbour[0][1], neighbour[1][0]:neighbour[1][1]]
        ssd = ((A-B)**2).sum()
        if (ssd < lowest_ssd):
            lowest_ssd = ssd
            match = neighbour
    if (match == coord or lowest_ssd < threshold):
        return None, -1
    return match, lowest_ssd

In [10]:
#HYPERPARAMETERS
block_width = 15
levels = 1
threshold = 55000

In [13]:
frame_load_path = './frames_A1/'
destination_path = './Motion/'

block_width = 15
frame1 = np.array(imread('./frames_A1/frame0.tif'))
blocks_frame = split_into_block_coords(frame1, block_width)


frame_counter = 0

while(1):
    img1 = np.array(imread(frame_load_path + 'frame%d.tif' % frame_counter))
    if img1 is None:
        print('No more frames to be loaded')
        break;
    try:
        img2 = np.array(imread(frame_load_path + 'frame%d.tif' % (frame_counter + 1)))
    except FileNotFoundError:
        print('No more frames to be loaded')
        break
    for i in range(len(blocks_frame)):
        current_block = [blocks_frame[i][1], blocks_frame[i][0]]
        neighbours = get_neighbour_coords(current_block, block_width, levels, frame1.shape[0], frame1.shape[1])
        match_coords, ssd = computing_min_ssd(neighbours, blocks_frame[i], img1, img2, block_width, threshold)
        if match_coords == None:
            continue
        else:
            #print("Draw arrow from ", blocks_frame[i], "to", match_coords)
            arrowdraw(img1, 
                    int((blocks_frame[i][1][1] + blocks_frame[i][1][0])/2),
                    int((blocks_frame[i][0][1] + blocks_frame[i][0][0])/2),
                    int((match_coords[1][1] + match_coords[1][0])/2),
                    int((match_coords[0][1] + match_coords[0][0])/2),
                      ssd
                    )
    imwrite(destination_path + 'frame%d.tif' % frame_counter, img1)
    frame_counter += 1    
         
path_to_output_video = './motion_estimation_video.mov'
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (int(frame_width), int(frame_height)))
frame_counter = 0
while(1):
    img = cv2.imread(destination_path + 'frame%d.tif' % frame_counter)
    if img is None:
        print('No more frames to be loaded')
        break;
    out.write(img)
    frame_counter += 1
print(frame_counter)
out.release()
cv2.destroyAllWindows()

No more frames to be loaded
No more frames to be loaded
449
